In [0]:
%run ./05_streaming_trigger

In [0]:
# Databricks notebook source
# MAGIC %run ./05-streaming-batch

# COMMAND ----------

class streamingBatchTestSuite():
    def __init__(self):
        self.base_data_dir = "/FileStore/data_spark_streaming_scholarnest"

    def cleanTests(self):
        print(f"Starting Cleanup...", end='')
        spark.sql("drop table if exists invoice_line_items")
        dbutils.fs.rm("/user/hive/warehouse/invoice_line_items", True)

        dbutils.fs.rm(f"{self.base_data_dir}/chekpoint/invoices", True)
        dbutils.fs.rm(f"{self.base_data_dir}/data/invoices", True)

        dbutils.fs.mkdirs(f"{self.base_data_dir}/data/invoices")
        print("Done")

    def ingestData(self, itr):
        print(f"\tStarting Ingestion...", end='')
        dbutils.fs.cp(f"{self.base_data_dir}/datasets/invoices/invoices_{itr}.json", f"{self.base_data_dir}/data/invoices/")
        print("Done")

    def assertResult(self, expected_count):
        print(f"\tStarting validation...", end='')
        actual_count = spark.sql("select count(*) from invoice_line_items").collect()[0][0]
        assert expected_count == actual_count, f"Test failed! actual count is {actual_count}"
        print("Done")

    def waitForMicroBatch(self, sleep=30):
        import time
        print(f"\tWaiting for {sleep} seconds...", end='')
        time.sleep(sleep)
        print("Done.")

    def runStreamTests(self):
        self.cleanTests()
        iStream = invoiceStreamBatch()
        streamQuery = iStream.process("30 seconds")

        print("Testing first iteration of invoice stream...") 
        self.ingestData(1)
        self.waitForMicroBatch()        
        self.assertResult(1253)
        print("Validation passed.\n")

        print("Testing second iteration of invoice stream...") 
        self.ingestData(2)
        self.waitForMicroBatch()
        self.assertResult(2510)
        print("Validation passed.\n") 

        print("Testing third iteration of invoice stream...") 
        self.ingestData(3)
        self.waitForMicroBatch()
        self.assertResult(3994)
        print("Validation passed.\n")

        streamQuery.stop()

    def runBatchTests(self):
        self.cleanTests()
        iStream = invoiceStreamBatch()

        print("Testing first batch of invoice stream...") 
        self.ingestData(1)
        self.ingestData(2)
        iStream.process("batch")
        self.waitForMicroBatch(30)        
        self.assertResult(2510)
        print("Validation passed.\n")

        print("Testing second batch of invoice stream...") 
        self.ingestData(3)
        iStream.process("batch")
        self.waitForMicroBatch(30)        
        self.assertResult(3994)
        print("Validation passed.\n")



# COMMAND ----------

sbTS = streamingBatchTestSuite()
sbTS.runStreamTests()	

# COMMAND ----------

sbTS.runBatchTests()

# COMMAND ----------

# MAGIC %md
# MAGIC &copy; 2021-2023 <a href="https://www.scholarnest.com/">ScholarNest Technologies Pvt. Ltd. </a>All rights reserved.<br/>
# MAGIC Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
# MAGIC Databricks, Databricks Cloud and the Databricks logo are trademarks of the <a href="https://www.databricks.com/">Databricks Inc</a>.<br/>
# MAGIC <br/>
# MAGIC <a href="https://www.scholarnest.com/privacy/">Privacy Policy</a> | <a href="https://www.scholarnest.com/terms/">Terms of Use</a> | <a href="https://www.scholarnest.com/contact-us/">Contact Us</a>


Starting Cleanup...Done
Starting Invoice Processing Stream...Done

Testing first iteration of invoice stream...
	Starting Ingestion...Done
	Waiting for 30 seconds...Done.
	Starting validation...Done
Validation passed.

Testing second iteration of invoice stream...
	Starting Ingestion...Done
	Waiting for 30 seconds...Done.
	Starting validation...Done
Validation passed.

Testing third iteration of invoice stream...
	Starting Ingestion...Done
	Waiting for 30 seconds...Done.
	Starting validation...Done
Validation passed.

Starting Cleanup...Done
Testing first batch of invoice stream...
	Starting Ingestion...Done
	Starting Ingestion...Done
Starting Invoice Processing Stream...Done

	Waiting for 30 seconds...Done.
	Starting validation...Done
Validation passed.

Testing second batch of invoice stream...
	Starting Ingestion...Done
Starting Invoice Processing Stream...Done

	Waiting for 30 seconds...Done.
	Starting validation...Done
Validation passed.

